In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import *
from keras.layers import *

# **Training the Model**

In [ ]:
path = ""
df = pd.read_csv(path)

In [ ]:
def get_old_examples(data, date):
    idxs = []
    date = 30*int(date[:2])+int(date[3:5])+365*int(date[-4:])
    for i in range(data["Date of Collection"].shape[0]):
        counter = data.index[i]
        date_str = data["Date of Collection"][counter]
        data_date = 30*int(date_str[:date_str.index("/")])+int(date_str[date_str.index("/")+1:date_str.rindex("/")])+365*int(date_str[-4:])
        if data_date <= date:
            idxs.append(i)
    display(data.iloc[idxs])
    return list(data.iloc[idxs].index)

In [ ]:
test_idxs = get_old_examples(df, "06/01/2019")

In [ ]:
# df.corr()

In [ ]:
df = df.iloc[:,5:-3]

In [ ]:
df.head()

In [ ]:
train_idxs = []
for i in range(df.shape[0]):
    if not i in test_idxs:
        train_idxs.append(i)

In [ ]:
len(test_idxs) + len(train_idxs) == df.shape[0]

In [ ]:
# train_idxs = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 60, 62, 65, 67, 68, 69, 70, 72, 74, 75, 76, 80, 81, 82, 83, 84, 85, 86, 87, 88, 91, 92, 93, 95, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 109, 111, 113, 114, 115, 116, 117, 118, 119, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 159, 160, 162, 163, 164, 165, 166]
# test_idxs = [20, 24, 26, 29, 42, 43, 44, 45, 46, 54, 59, 61, 63, 64, 66, 71, 73, 77, 78, 79, 89, 90, 94, 96, 102, 110, 112, 120, 121, 122, 132, 156, 157, 158, 161]

In [ ]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values
Y = np.log10(Y)

In [ ]:
X_t = X[train_idxs]
X_v = X[test_idxs]
y_t = Y[train_idxs]
y_v = Y[test_idxs]

In [ ]:
print(X_t.shape, X_v.shape, y_t.shape, y_v.shape)

In [ ]:
means = np.mean(X_t, axis=0, keepdims=True)
stds = np.std(X_t, axis=0, keepdims=True)

X_t = (X_t - means) / stds
X_v = (X_v - means) / stds

In [ ]:
model = Sequential([
    Dense(64, input_shape=(X_t.shape[1],), activation="relu"),
    Dropout(0.2),
    Dense(64, activation="relu"),
    Dropout(0.2),
    Dense(64, activation="relu"),
    Dropout(0.2),
    Dense(64, activation="relu"),
    Dropout(0.2),
    Dense(64, activation="relu"),
    Dropout(0.2),
    Dense(64, activation="relu"),
    Dense(1, activation="relu")
])

In [ ]:
# model.summary()

In [ ]:
model.compile(optimizer="adam", loss="mae", metrics=["mse"])

In [ ]:
path = ""
# model.save_weights(path)
# model.load_weights(path)

In [ ]:
history = model.fit(X_t, y_t, validation_data=(X_v, y_v), batch_size=8, epochs=150)

In [ ]:
plt.plot(pd.DataFrame(history.history).loss)
plt.plot(pd.DataFrame(history.history).val_loss)
plt.show()

In [ ]:
y_t_pred = model.predict(X_t)
y_v_pred = model.predict(X_v)

In [ ]:
plt.scatter(y_t_pred, y_t)
plt.show()

In [ ]:
plt.scatter(y_v_pred, y_v)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,4))
ax[0].scatter(y_t_pred, y_t)
ax[0].set_title("Training Data", fontsize=14)
ax[0].set_yticklabels(ax[0].get_yticks(), fontsize=12)
ax[0].set_xticklabels(ax[0].get_xticks(), fontsize=12)
ax[0].set_xlabel("Predicted Larvae Count (Log)", fontsize=12)
ax[0].set_ylabel("True Larvae Count (Log)", fontsize=12)

ax[1].scatter(y_v_pred, y_v)
ax[1].set_title("Historical Validation Data (Before June 2019)", fontsize=14)
ax[1].set_yticklabels(ax[1].get_yticks(), fontsize=12)
ax[1].set_xticklabels(ax[1].get_xticks(), fontsize=12)
ax[1].set_xlabel("Predicted Larvae Count (Log)", fontsize=12)
ax[1].set_ylabel("True Larvae Count (Log)", fontsize=12)
plt.subplots_adjust(wspace=0.25)
plt.show()

In [ ]:
m, b, r, p, SE, = linregress(10**y_v, 10**y_v_pred.reshape(-1))
print(r, p/2)

In [ ]:
plt.scatter(10**y_v_pred, 10**y_v)
plt.ylim(0,40)
plt.xlim(0,40)
plt.show()

In [ ]:
m, b, r, p, SE, = linregress(10**y_t, 10**y_t_pred.reshape(-1))
print(r, p)

# **Regions of Interest**


In [ ]:
lr = LinearRegression()
lr.fit(df["Average Precipitation (in.)"].values.reshape(-1,1), df["Days with Precipitation"].values.reshape(-1,1))
days_precip_pred = lr.predict(df["Average Precipitation (in.)"].values.reshape(-1,1))

In [ ]:
plt.scatter(df["Average Precipitation (in.)"], df["Days with Precipitation"])
plt.plot(df["Average Precipitation (in.)"].values.reshape(-1), days_precip_pred.reshape(-1), c="r")
plt.show()

In [ ]:
# Reading in Predictions for USA
path = ""
us_mean_temp_predictions = pd.read_csv(path)
us_max_temp_predictions = pd.read_csv(path)
us_min_temp_predictions = pd.read_csv(path)
us_precip_predictions = pd.read_csv(path)

In [ ]:
us_elevation = np.array([451.861,4213.116,524.218,2796.362,6860.608,464.203,44.569,58.263,498.368,5231.242,618.707,
                         738.93,1062.981,1906.834,815.661,93.497,757.451,399.999,534.869,771.489,1198.058,282.345,
                         844.716,4134.565,2604.784,5527.712,1121.307,227.948,5771.761,1045.326,744.65,1827.502,
                         891.436,1287.858,3520.85,1233.219,248.131,316.708,2184.603,938.148,1649.2,6011.909,
                         1209.995,993.246,2280.594,1657.547,1006.093,6396.121] * 3).reshape(3,48).T.reshape(-1)
us_elevation /= 3.28084

In [ ]:
names_us = ['Alabama','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia',
 'Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan',
 'Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico',
    'New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island',
            'South Carolina','South Dakota','Tennessee','Texas','Utah',
    'Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']

In [ ]:
# Reading in Predictions for Texas
path = ""
tx_mean_temp_predictions = pd.read_csv(path)
tx_max_temp_predictions = pd.read_csv(path)
tx_min_temp_predictions = pd.read_csv(path)
tx_precip_predictions = pd.read_csv(path)

In [ ]:
tx_elevation = np.array([100, 600, 1100, 500, 20, 350, 120, 180, 1300, 20] * 4).reshape(4,10).T.reshape(-1)

In [ ]:
names_tx = ["East Texas","Edwards Plateau","High Plains","Low Rolling Plains","Lower Valley",
               "North Central","South Central","South Texas","Trans Pecos","Upper Coast"]

In [ ]:
def get_predictions(mean_temp, max_temp, min_temp, precip, elevation, names, num_locations, num_timesteps):
    days_precip = lr.predict(precip.iloc[:,1:-1].values.flatten().reshape(-1,1)).reshape(precip.shape[0],num_timesteps)
    days_precip = np.round(days_precip).flatten()
    mean_temp = mean_temp.iloc[:,1:-1].values.flatten()
    max_temp = max_temp.iloc[:,1:-1].values.flatten()
    min_temp = min_temp.iloc[:,1:-1].values.flatten()
    precip = precip.iloc[:,1:-1].values.flatten()
    test = np.vstack((max_temp,mean_temp,min_temp,days_precip,precip,elevation)).T
    test = (test - means) / stds
    pred = model.predict(test)
    pred_df = pd.DataFrame(pred.reshape(num_locations,num_timesteps))
    pred_df["Location"] = names
    pred_df.iloc[:,:-1] = 10 ** pred_df.iloc[:,:-1].values
    return pred_df

In [ ]:
us_pred = get_predictions(us_mean_temp_predictions,
                          us_max_temp_predictions,
                          us_min_temp_predictions,
                          us_precip_predictions,
                          us_elevation,
                          names_us,
                          48, 3)

In [ ]:
tx_pred = get_predictions(tx_mean_temp_predictions,
                          tx_max_temp_predictions,
                          tx_min_temp_predictions,
                          tx_precip_predictions,
                          tx_elevation,
                          names_tx,
                          10, 4)

In [ ]:
path = ""
us_pred_df.to_csv(path)
tx_pred_df.to_csv(path)